## An end-to-end pipeline for the analysis of HCP diffusion MRI data using Dipy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
%matplotlib inline

### Get the data from AWS S3

We assume that you have a file '.aws/credentials', that includes a section: 

    [hcp]
    AWS_ACCESS_KEY_ID=XXXXXXXXXXXXXXXX
    AWS_SECRET_ACCESS_KEY=XXXXXXXXXXXXXXXX
with the credentials you got [from HCP](https://wiki.humanconnectome.org/display/PublicData/How+To+Connect+to+Connectome+Data+via+AWS)

In [2]:
data_files = {'./bvals':'HCP/994273/T1w/Diffusion/bvals', 
              './bvecs':'HCP/994273/T1w/Diffusion/bvecs', 
              './data.nii.gz':'HCP/994273/T1w/Diffusion/data.nii.gz'}

In [3]:
import botocore.session
import boto3
boto3.setup_default_session(profile_name='hcp')
s3 = boto3.resource('s3')
bucket = s3.Bucket('hcp-openaccess')

In [4]:
for k in data_files.keys():
    if not op.exists(k):
        bucket.download_file(data_files[k], k)


In [5]:
import dipy.core.gradients as dpg
import nibabel as nib

In [6]:
# Read in the gradient table, setting the threshold to 10, so that low values are considered as 0:
gtab = dpg.gradient_table('./bvals', './bvecs', b0_threshold=10)

In [7]:
img = nib.load('./data.nii.gz')

In [8]:
data = img.get_data()

### Brain extraction (median Otsu)

See (example and explanations)

In [9]:
# We look at the average non diffusion-weighted as data for brain extraction
mean_b0 = np.mean(data[..., gtab.b0s_mask], -1)

In [10]:
if not op.exists('./mask.nii.gz'):
    from dipy.segment.mask import median_otsu
    _, mask = median_otsu(mean_b0, 4, 2, False, vol_idx=np.where(gtab.b0s_mask), dilate=1)
    nib.save(nib.Nifti1Image(mask.astype(int), img.affine), 'mask.nii.gz')
else: 
    mask = nib.load('mask.nii.gz').get_data()

### Denoising (NLMEANS)

See [example and explanations](http://nipy.org/dipy/examples_built/denoise_nlmeans.html#example-denoise-nlmeans)


In [14]:
if not op.exists('./denoised_data.nii.gz'):
    from dipy.denoise import nlmeans
    from dipy.denoise.noise_estimate import estimate_sigma
    sigma = estimate_sigma(data)
    denoised_data = nlmeans.nlmeans(data, sigma=sigma, mask=mask)
    nib.save(nib.Nifti1Image(denoised_data, img.affine), 'denoised_data.nii.gz')
else:
    denoised_data = nib.load('denoised_data.nii.gz').get_data()

###  Registration to a template

In [15]:
from dipy.data import read_mni_template

In [16]:
MNI_T2 = read_mni_template()

Data size is approximately 35MB
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/mni_template 


In [17]:
MNI_T2_data = MNI_T2.get_data()

In [18]:
MNI_T2_affine = MNI_T2.get_affine()

In [19]:
from dipy.align.metrics import CCMetric
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration

In [22]:
use_metric = CCMetric(3, sigma_diff=2.0)

sdr = SymmetricDiffeomorphicRegistration(use_metric, [10, 10, 5],
                                         step_length=0.25)

mapping = sdr.optimize(MNI_T2_data, mean_b0,
                       static_grid2world=img.affine,
                       moving_grid2world=MNI_T2_affine,
                       prealign=None)

xformed_b0 = mapping.transform(mean_b0)

Creating scale space from the moving image. Levels: 3. Sigma factor: 0.200000.
Creating scale space from the static image. Levels: 3. Sigma factor: 0.200000.
Optimizing level 2
Optimizing level 1
Optimizing level 0


### Modeling (DTI)

See [example and explanations](http://nipy.org/dipy/examples_built/reconst_dti.html#example-reconst-dti)

In [23]:
import dipy.reconst.dti as dti

ten_model = dti.TensorModel(gtab)
ten_fit = ten_model.fit(data, mask=mask)

nib.save(nib.Nifti1Image(ten_fit.fa, img.affine), 'dti_fa.nii.gz')
nib.save(nib.Nifti1Image(ten_fit.fa, img.affine), 'dti_md.nii.gz')


### Segmentation (FA)

### Modeling (CSD)

### Tracking (Probabilistic/Deterministic)